In [1]:
#GPU版本 指定在CPU上运行
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import pandas as pd

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 数据预处理

## panda数据读取

In [3]:
train_data=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/train.csv",header=None,usecols=[1,2])
train_labels=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/train.csv",header=None,usecols=[0])
test_data=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/test.csv",header=None,usecols=[1,2])
test_labels=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/test.csv",header=None,usecols=[0])

## titlte+description

In [4]:
#array
train_data=train_data.values
y_train=train_labels.values
test_data=test_data.values
y_test=test_labels.values

train_messages = [None]*120000
test_messages = [None]*7600

for index, data in enumerate(train_data):
    train_messages[index] = data[0]+'.'+data[1]

for index, data in enumerate(test_data):
    test_messages[index] = data[0]+'.'+data[1]
    
for index,data in enumerate(y_train):
    y_train[index][0]=data[0]-1
for index,data in enumerate(y_test):
    y_test[index][0]=data[0]-1

## tokenizer词典 

In [5]:
tokenizer=keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                                   lower=True, split=' ', 
                                   char_level=False, 
                                   oov_token=None)
tokenizer.fit_on_texts(train_messages)#词典

In [6]:
#print( 'word_counts:',tokenizer.word_counts) #单词出现次数
#print( '\nword_index:',tokenizer.word_index) #单词出现的位置
#print( '\nword_docs:',tokenizer.word_docs) #单词出现在的文档（行）数
#print( '\nindex_docs',tokenizer.index_docs) #word_index：word_docs

In [7]:
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)
maxlen=0
max_index=0
for i, seq in enumerate(train_sequences):
    if len(seq)>maxlen:
        maxlen=len(seq)
    if max(seq)>max_index:
        max_index=max(seq)
print(maxlen)
print(max_index)

181
70344


In [8]:
train_x = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen)
test_x = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen)
y_train_labels = keras.utils.to_categorical(y_train, num_classes=4)
y_test_labels = keras.utils.to_categorical(y_test, num_classes=4)
train_x
#print( tokenizer.texts_to_matrix(test_messages))  # 矩阵化=one_hot

array([[    0,     0,     0, ...,  4049,   797,   332],
       [    0,     0,     0, ...,     4,     1,   128],
       [    0,     0,     0, ...,     1,  1214, 14993],
       ...,
       [    0,     0,     0, ...,   346,    65,   123],
       [    0,     0,     0, ...,    42,    16,  1666],
       [    0,     0,     0, ...,  2095,  3435,    72]], dtype=int32)

# basic cnn model

In [9]:
def base_cnn_model(y,max_features,embedding_dims,filters):
    kernel_size = 3
    model = keras.Sequential()
    model.add(keras.layers.Embedding(max_features, embedding_dims)) # 使用Embedding层将每个词编码转换为词向量(10000,181,128)
    model.add(keras.layers.Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # 池化
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(y.shape[1], activation='softmax')) #第一个参数units: 全连接层输出的维度，即下一层神经元的个数。
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()

    return model

In [10]:
max_features=70345
embedding_dims=128
filters=32
#y_train=np_utils.to_categorical(4)
model=base_cnn_model(y_train_labels,max_features,embedding_dims,filters)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         9004160   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          12320     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 9,016,612
Trainable params: 9,016,612
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(train_x,
                    y_train_labels,
                    validation_split=0.2,
                    batch_size=64,
                    epochs=30,
                    verbose=1,
                    shuffle=True)

Train on 96000 samples, validate on 24000 samples
Epoch 1/30
96000/96000 [==============================] - 231s 2ms/step - loss: 1.0290 - acc: 0.5327 - val_loss: 0.5861 - val_acc: 0.8195
Epoch 2/30
96000/96000 [==============================] - 298s 3ms/step - loss: 0.6987 - acc: 0.7193 - val_loss: 0.4613 - val_acc: 0.8587
Epoch 3/30
96000/96000 [==============================] - 291s 3ms/step - loss: 0.5922 - acc: 0.7806 - val_loss: 0.3964 - val_acc: 0.8727
Epoch 4/30
96000/96000 [==============================] - 293s 3ms/step - loss: 0.5395 - acc: 0.8011 - val_loss: 0.3808 - val_acc: 0.8802
Epoch 5/30
96000/96000 [==============================] - 289s 3ms/step - loss: 0.5039 - acc: 0.8141 - val_loss: 0.3805 - val_acc: 0.8765
Epoch 6/30
96000/96000 [==============================] - 286s 3ms/step - loss: 0.4785 - acc: 0.8219 - val_loss: 0.4020 - val_acc: 0.8734
Epoch 7/30
96000/96000 [==============================] - 284s 3ms/step - loss: 0.4609 - acc: 0.8275 - val_loss: 0.3762 - 

In [12]:
results = model.evaluate(test_x, y_test_labels)
print(results)

7600/7600 [==============================] - 2s 206us/step
[0.5340392661408374, 0.8772368421052632]
